<a href="https://colab.research.google.com/github/PadmanagasriGundubogula/FMML-LAB-1/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

###1answer
The accuracy of the validation set can be affected when you change the percentage of data allocated to it in several ways:

1. **Increasing the Percentage of Validation Set**:

   - **Pros**:
     - **Higher Confidence**: With a larger validation set, you get a better estimate of how well your model generalizes to unseen data. This can give you higher confidence in your model's performance.
     - **Better Generalization**: A larger validation set can help you detect overfitting more effectively. It's more likely to expose issues where your model performs well on the training data but poorly on new, unseen data.

   - **Cons**:
     - **Reduced Training Data**: Increasing the validation set percentage reduces the amount of data available for training. This can be problematic if you have limited data to begin with, as it may lead to underfitting, where the model doesn't learn the underlying patterns in the data well.

2. **Reducing the Percentage of Validation Set**:

   - **Pros**:
     - **More Data for Training**: By allocating less data to the validation set, you have more data available for training. This can be advantageous, especially if you have a small dataset, as it allows your model to potentially learn better.

   - **Cons**:
     - **Less Reliable Evaluation**: With a smaller validation set, your estimate of the model's performance on unseen data becomes less reliable. The validation set may not adequately represent the diversity and complexity of the real-world data, leading to overfitting.

The choice of the percentage of data allocated to the validation set is a trade-off between obtaining a reliable estimate of model performance and maximizing the amount of data available for training. It often depends on the specific problem, the size of your dataset, and the computational resources at your disposal. In practice, it's common to use techniques like cross-validation to mitigate the impact of this trade-off and obtain a more robust estimate of a model's performance.

###2nd anwer

The size of the training and validation sets can have a significant impact on how well you can predict the accuracy on the test set using the validation set. This relationship is often referred to as the bias-variance trade-off, and it plays a crucial role in machine learning model development.

1. **Large Training Set**:
   - When you have a large training set, your model has more data to learn from, which can help it capture complex patterns in the data. A larger training set can lead to a model with lower bias (i.e., it's better at fitting the training data) and potentially higher accuracy.
   - However, with a very large training set, you might need more computational resources and time to train the model, and there's a risk of overfitting, where the model becomes too specialized to the training data and doesn't generalize well to unseen data.

2. **Large Validation Set**:
   - A larger validation set can provide a more reliable estimate of your model's performance because it tests the model on a wider range of data. This can help you detect overfitting early and make better decisions about model hyperparameters and architecture.
   - However, if your validation set is too large relative to your training set, you may not be effectively using your available data for training. This could result in underfitting, where the model fails to capture important patterns in the training data.

3. **Small Validation Set**:
   - A small validation set can lead to more variability in your estimated model performance. This is because the validation set's performance is more sensitive to the specific samples within it.
   - It might also be less effective at detecting overfitting, as it may not provide a representative assessment of your model's generalization to unseen data.

In practice, it's essential to strike a balance between the sizes of the training and validation sets based on the available data. Common techniques include using techniques like cross-validation, where the data is divided into multiple folds for both training and validation, or using techniques like stratified sampling to ensure that both sets are representative of the overall dataset.

The choice of validation set size should depend on your specific problem, the amount of data you have, and computational resources available. Experimenting with different split ratios and techniques is often necessary to find the right balance and achieve a model that generalizes well to unseen data.

###3rd answer
The percentage you should reserve for the validation set when training a machine learning model depends on several factors, and there isn't a one-size-fits-all answer. The balance between the size of your training set and validation set depends on the size of your overall dataset, the complexity of your model, and the amount of data you have available.

Here are some general guidelines to help you decide:

1. **Rule of Thumb:** A common starting point is to reserve 70-80% of your data for training and the remaining 20-30% for validation. This is a good rule of thumb and often works well for moderately sized datasets.

2. **Cross-Validation:** If you have a relatively small dataset, you might consider using techniques like k-fold cross-validation. In k-fold cross-validation, you split your data into k subsets (folds) and perform training and validation k times, using a different fold as the validation set in each iteration. This helps ensure that your model generalizes well.

3. **Data Abundance:** If you have a very large dataset, you can afford to allocate a smaller percentage to the validation set because you still have a substantial amount of data for training. In such cases, you might reserve only 10-20% for validation.

4. **Imbalanced Datasets:** If your dataset is highly imbalanced (one class vastly outnumbering the others), you may need to use techniques like stratified sampling to ensure that both training and validation sets have representative samples of each class.

5. **Hyperparameter Tuning:** If you plan to perform extensive hyperparameter tuning or model selection, you might want to set aside a larger validation set to get a more reliable estimate of model performance.

6. **Time Series Data:** If you are working with time series data, you should typically use a time-based split. For example, you might reserve the most recent data for validation to simulate real-world usage.

7. **Domain Knowledge:** Your domain knowledge and understanding of the problem can also influence the split. If you know that certain data points are more critical for validation due to their real-world importance, you might adjust the split accordingly.

In practice, it's often a good idea to experiment with different splits and use techniques like cross-validation to evaluate how well your model performs with different training and validation set sizes. The key is to strike a balance that allows your model to learn from enough data while also having a reliable validation set to assess its performance. Ultimately, the choice of the validation set percentage should be guided by the specific characteristics of your dataset and the goals of your machine learning project.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


###1st answer
Yes, averaging the validation accuracy across multiple splits can often give more consistent and reliable results when evaluating the performance of a machine learning model compared to relying on a single validation split. This practice is known as cross-validation.

Cross-validation involves splitting your dataset into multiple subsets, typically called "folds," and then training and evaluating your model multiple times, each time using a different fold as the validation set and the remaining folds as the training set. The validation accuracy is recorded for each iteration, and the final evaluation metric is computed by averaging these individual results.

Here are a few reasons why cross-validation can provide more consistent results:

1. Reduced Variance: By using multiple validation sets, cross-validation helps reduce the variance in your performance estimate. A single random split can sometimes lead to overfitting or underestimating the model's true performance due to the specific data points included in that split. Cross-validation averages out these variations.

2. Better Generalization: Cross-validation provides a more robust estimate of a model's generalization performance because it evaluates the model's performance on different subsets of the data. This can help you determine how well your model performs across different parts of your dataset, which is crucial for assessing its overall reliability.

3. Increased Confidence: Averaging the results from multiple splits provides a more stable estimate of model performance. You can also compute confidence intervals or standard deviations to understand the level of certainty associated with your performance metric.

Common cross-validation techniques include k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation, among others. The choice of the specific cross-validation technique may depend on the size of your dataset and the nature of your problem.

In summary, using cross-validation and averaging the validation accuracy across multiple splits is a good practice to obtain more consistent and reliable estimates of a model's performance, which can be particularly important for making informed decisions about model selection and hyperparameter tuning.

###2nd asnwer
The accuracy of a test estimate depends on various factors, including the methodology used, the quality and quantity of data, and the complexity of the model or system being tested. The phrase "it" in your question is not specific, so I'll provide some context.

If you're referring to a machine learning model or algorithm, then the accuracy of the test estimate can be influenced by factors such as the choice of evaluation metric, the size and representativeness of the test dataset, and the model's training process. Generally, using a larger and more diverse test dataset can lead to a more accurate estimate of a model's performance on unseen data. Additionally, using appropriate evaluation metrics (e.g., accuracy, precision, recall, F1 score) can provide a more comprehensive understanding of how well a model generalizes to new data.

If you're referring to a specific technology or tool designed to estimate test accuracy, it would be helpful to provide more context or specify what you're asking about so I can provide a more tailored answer.

In summary, the accuracy of a test estimate depends on multiple factors, and improving it often involves using best practices in data collection, model training, and evaluation.

###3rd answer
The effect of the number of iterations on an estimate depends on the specific algorithm or method being used. In general, increasing the number of iterations can have different effects on the estimate, and whether it leads to a better estimate or not depends on various factors:

1. Convergence: Many iterative algorithms are designed to converge to a solution. As you increase the number of iterations, the estimate may get closer to the true value, improving the accuracy of the estimate. However, there is usually a point of diminishing returns where further iterations do not significantly improve the estimate.

2. Computational Resources: Increasing the number of iterations often requires more computational resources (e.g., time and memory). Therefore, there may be practical limitations to how many iterations you can perform, and you'll need to balance accuracy with computational cost.

3. Convergence Rate: Some iterative algorithms converge rapidly in the early iterations and then slow down as they approach a solution. In such cases, increasing iterations beyond a certain point may not lead to a significantly better estimate.

4. Noise and Uncertainty: In some cases, increasing the number of iterations can make an estimate more sensitive to noise or uncertainty in the data, leading to less robust results.

5. Algorithm-specific Considerations: Different algorithms have different behaviors with respect to the number of iterations. Some algorithms may require a fixed number of iterations to achieve a desired level of accuracy, while others may benefit from more iterations up to a certain point.

6. Initial Conditions: The choice of initial conditions or starting point can also impact the effect of iterations. Sometimes, starting closer to the true solution can reduce the need for a large number of iterations.

In summary, there is no universal rule that more iterations always lead to a better estimate. The impact of the number of iterations on the estimate depends on the specific problem, algorithm, and the trade-off between computational resources and accuracy. It's often necessary to perform sensitivity analyses or validation experiments to determine the appropriate number of iterations for a given task.